This is a tutorial pyflowline notebook.

This tutorial is an example of the pyflowline application using an MPAS mesh.

First, let's load some Python libraries 

In [1]:
#step 1
import os
import sys
from pathlib import Path
from os.path import realpath

Now let's define the path of the package

In [5]:
#step 2 option A
sPath_notebook = Path().resolve()
sPath_parent = str(Path().resolve().parents[0]) # data is located two dir's up
print(sPath_notebook)
print(sPath_parent)



/Users/liao313/workspace/python/pyflowline/notebooks
/Users/liao313/workspace/python/pyflowline


If this parent folder is where your pyflowline source code is installed, we can continue. If not, you can manually specify it such as:

In [6]:
sPath_parent = '/Users/liao313/workspace/python/pyflowline'

Now append this path to the Python path.

In [7]:
sys.path.append(sPath_parent)

The pyflowline python package uses the OOP approach to manage configuration settings.
Set up the case object. 
Set up some core keywords.

In [9]:
#step 2 option B
import pyflowline
#load the read configuration function
from pyflowline.pyflowline_read_model_configuration_file import pyflowline_read_model_configuration_file

sPath_parent = str(Path().resolve().parents[0]) 
print(sPath_parent)


/Users/liao313/workspace/python/pyflowline


We provide two options to set up a simulation case. First, from scratch (iFlag_option: 1); second, using a json file (iFlag_option = 2). For details of the json file format, please read the documentation.
We will use option 1 in this tutorial.

In [12]:
#step 3
#set up workspace
sWorkspace_data = realpath( sPath_parent +  '/data/susquehanna' )
sWorkspace_input =  str(Path(sWorkspace_data)  /  'input')
sWorkspace_output=  str(Path(sWorkspace_data)  /  'output')     

In [18]:
#step 4A
#pyflowline use a json file to organize all the input information, and an example json file is provided. 
sFilename_configuration_in = realpath( sPath_parent +  '/examples/susquehanna/pyflowline_susquehanna_mpas.json' )
#if the configuration file does not exist, you need to check the path setting
if os.path.isfile(sFilename_configuration_in):
    pass
else:
    print('This configuration does not exist: ', sFilename_configuration_in )
    exit

In [14]:
#step 4B
#we can check what is the content of this json file
import json
with open(sFilename_configuration_in, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed, indent=4))

{
    "sFilename_model_configuration": "/qfs/people/liao313/workspace/python/pyflowline/pyflowline/config/hexwatershed_susquehanna_mpas.json",
    "sWorkspace_data": "/people/liao313/data",
    "sWorkspace_output": "/compyfs/liao313/04model/pyflowline/susquehanna",
    "sWorkspace_project": "/hexwatershed/susquehanna",
    "sWorkspace_bin": "/people/liao313/bin",
    "sRegion": "susquehanna",
    "sModel": "pyflowline",
    "sJob": "hex",
    "iFlag_standalone": 1,
    "iFlag_create_mesh": 1,
    "iFlag_save_mesh": 1,
    "iFlag_simplification": 1,
    "iFlag_intersect": 1,
    "iFlag_flowline": 1,
    "iFlag_use_mesh_dem": 1,
    "iFlag_global": 0,
    "iFlag_multiple_outlet": 0,
    "iFlag_rotation": 0,
    "iCase_index": 1,
    "iMesh_type": 1,
    "dLongitude_left": -79,
    "dLongitude_right": -74.5,
    "dLatitude_bot": 39.2,
    "dLatitude_top": 42.8,
    "dResolution_degree": 5000,
    "dResolution_meter": 5000,
    "sDate": "20220110",
    "sMesh_type": "mpas",
    "sFilename_

The meaning of these json keywords are explained in the pyflowline documentation: https://pyflowline.readthedocs.io/en/latest/data/data.html#inputs

For some parameters, we can change them using the following function call.

For some other parameters (e.g., path to file), you need to modify the json file using a text editor.

If the function returns an error, you should update the json file(s). 

In [25]:
#step 5
sMesh_type = 'mpas'
iCase_index = 1
dResolution_meter=5000
sDate='20230101'
oPyflowline = pyflowline_read_model_configuration_file(sFilename_configuration_in, iCase_index_in=iCase_index, 
                                                    sMesh_type_in= sMesh_type, sDate_in=sDate)

You can review the setting again.

In [ ]:
print(oPyflowline.tojson())

After the case object was created, we can run the mainly three types of operations. 
First, we can preprocess the flowline.

Before the operation, we can visualize the original or raw flowline dataset. For most visualization, all you need to provide is a keyword, a look up table is provided in the documentation.
Be default, all the dataset will be converted to the WGS84 system to maintain consistency.

You can also change the outlet coordinates.

In [ ]:
oPyflowline.aBasin[0].dLatitude_outlet_degree=39.4620
oPyflowline.aBasin[0].dLongitude_outlet_degree=-76.0093


In [ ]:
#setup the model       
oPyflowline.setup()

In [ ]:
#run step 1
oPyflowline.flowline_simplification()


After the preprocess, we can visualize the final flowline 

Second, we can creata a mesh

In [ ]:
#run step 2
oPyflowline.dLongitude_left= -79
oPyflowline.dLongitude_right= -74.5
oPyflowline.dLatitude_bot= 39.20
oPyflowline.dLatitude_top= 42.8

aCell = oPyflowline.mesh_generation()

We can also visualize the mesh:

Last, we can operate the intersect to generate desired flowlines. But we recommend that we run the intersect with some postprocess.

In [ ]:
#runs step 3
oPyflowline.reconstruct_topological_relationship(aCell)


After this step, we can now visualize the final flowline again.

In [ ]:
#export output

oPyflowline.export()

In [ ]:
print('Finished')

